# MLPモデルとPreprocessing
## 前回行ったこと
 - 交差検証
 - グリッドサーチによるハイパーパラメータチューニング

## 今回行うこと
- MLP（多層パーセプトロン）モデルを動かす
- 特徴ベクトルに対する学習前の処理
- 付録：EDA（Exploratory Data Analysis, 探索的データ分析）

## （復習）データの読み込み・特徴ベクトルの構築
one-hotエンコーディングを用いた特徴ベクトルを再び作ります．
もう詳しく説明することはしません．
全て一つのセルにまとめました．
詳細は前回の資料を参照してください．

In [ ]:
import numpy as np
import pandas as pd 

# Google Colabの場合
from google.colab import drive
drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）
d_train = pd.read_csv("drive/My Drive/data/train.csv") 
d_test = pd.read_csv("drive/My Drive/data/test.csv") 

# 自身のPython環境で動かしている場合，コメントアウトを外す
#d_train = pd.read_csv("data/train.csv")
#d_test = pd.read_csv("data/test.csv")

print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")

# targetの値
y_train = d_train.pop('stroke')
y_train = y_train.values # numpyのarrayに変換
print(y_train)

# one-hot encoding
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["gender","ever_married", "work_type", "Residence_type", "smoking_status"] # カテゴリカル変数の列名
d_train_test = pd.get_dummies(d_train_test, columns=columns_cat) # get_dummiesを使ってone-hotエンコーディング
d_train_test.pop("bmi") # 今回はbmiデータを使わず，捨てる
d_train = d_train_test[:n_train] # d_train_test_onehotの訓練データ部分
d_test = d_train_test[n_train:] # d_train_test_onehotのテストデータ部分

X_train = d_train.values # np.arrayに変換
X_test = d_test.values  # np.arrayに変換

いい加減しつこい気もしますが，復習と予測結果の比較のために`LogisticRegression`を動かしておきます．
ハイパーパラメータ（学習するのではなく，ユーザが決める要素）はここでは`fit_intercept=False`，`class_weight='balanced'`とします．

In [ ]:
from sklearn.linear_model import LogisticRegression # LogisticRegressionを使えるようにする
lr = LogisticRegression(fit_intercept=False, class_weight='balanced') # インスタンスの作成
lr.fit(X_train, y_train)
y_pred_test_lr = lr.predict_proba(X_test)[:,1]
print(y_pred_test_lr)

### 多層パーセプトロン（Multi-layer perceptron，MLP）を動かす
今回は，[多層パーセプトロン（Multi-layer perceptron, MLP）](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)を動かしてみます．
**MLPはニューラルネットワーク**の一種で，多層のニューラルネットワークの中では最もベーシックなものです．
MLPは，数理的には，
1. ベクトルに行列を掛ける
2. 活性化関数と呼ばれる非線形な関数を噛ませる

を繰り返したモデルです．
MLPをノード（点）とエッジ（線）を用いてグラフィカルに表現すると，上の2つの演算を繰り返した回数だけ，入力と出力の間に「層」があるように見えます．
そのため，上の2つの演算を行う回数を中間層（隠れ層）の数と言い，掛ける行列のことを中間層の重み（行列）と呼びます．
中間層の重み行列がMLPの学習するパラメータで，**勾配法**と呼ばれる方法で学習を行います．
勾配法（とくに，ここでは誤差関数の最小化を考えることにするので，勾配降下法）は，簡単に説明すると以下のような反復を行うアルゴリズムです：

1. パラメータ$\mathbf{\Theta}$の初期値を定め，以下の2と3を収束するまで繰り返す．
2. 最小化したい関数$L$に対するパラメータの勾配$\nabla L(\mathbf{\Theta})$を計算する．ニューラルネットワークにおいては，この勾配の計算を **誤差逆伝播法（Backpropagation）** と呼ばれる方法で効率よく行う．
3. 2で求めた勾配を使ってパラメータを少し動かす．最も単純な方法（＝厳密に単に勾配降下法と言った場合は）では以下のように動かす：
    $$\mathbf{\Theta} \leftarrow \mathbf{\Theta} - \eta \nabla L(\mathbf{\Theta}),$$
ここで，$\eta > 0$は学習率やステップサイズと呼ばれるハイパーパラメータで，一回の反復でどの程度パラメータを動かすかを表す．

一応簡単に説明しましたが，これでは不十分・わかりにくいように思います．
詳細は，例えば[scikit-learnの解説](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#neural-networks-supervised)や，[授業で使っている教科書](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf)や，その他多数の書籍やWeb上の記事を参考にしてみてください（「人工知能」の授業で触れているのではないかとも思いますが）．

では実際に使ってみます．
まずはimportですが，分類のためのMLPは`MLPClassifier`という名前で，`sklearn.neural_network`の中にあります．

In [ ]:
from sklearn.neural_network import MLPClassifier

それではとりあえず，デフォルトのパラメータで使ってみましょう．
デフォルト設定（チューニングをしていない）のモデルの評価をするために，`cross_validate`をimportします．
また，後のため，`GridSearchCV`もimportします．

In [ ]:
from sklearn.model_selection import cross_validate, GridSearchCV

前回と同様に，`cross_validate`で評価をしてみます．
`cross_validate`には，学習・評価をしたいモデルとデータ，そして分割数とスコア関数（の名前）を与えるのでした．
前回同様，分割数`cv=5`とし，スコア関数は`"arc_roc"`とします．
`cross_validate`を動かすと`scores`という辞書オブジェクトが返ってきます．
`"test_score"`というキーで交差検証のスコアを取得できるのでした．
5回のスコアとその平均を最後に`print`します．
結局，以下のようになります（前回のほぼコピペ）．

In [ ]:
mlp = MLPClassifier()
scores = cross_validate(mlp, X_train, y_train, cv=5, 
                        scoring="roc_auc")

print(f"交差検証の5回のスコア：{scores['test_score']}")
print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}")

少し待った時間がかかった結果，**トンデモナイスコア**が返ってきました，何だこれ！？
過学習にしてもひどいですね．

ひどい過学習をしているのかなと思いつつ，一応，**訓練データに対するスコア**を確認してみます．
「訓練データに対するスコアを参考にするな」と以前の資料では言ったように思いますが，訓練データに対するスコアは以下のように使うことができます：**訓練データに対するスコアも悪い場合，そもそも学習ができていない**ので，**プログラムの使い方を大きく間違っている**・**そもそもプログラムにバグがある**（今回は考えにくいですが，自身でコアのアルゴリズムを実装した場合はあり得ます）と言ったことを検出できます（つまり，**過学習以前の問題**が起こっている）．
この場合，ドキュメントを注意深く読み直す必要があります．

`cross_validate`では，`return_train_score`という引数を`True`にすることで，訓練時のスコアも取得できます．
`train_score`というキーでアクセスできます（`cross_validate`は辞書型オブジェクトを返すのでしたね）．
ちょっと面倒ですが，もう一度交差検証をしてみます．

In [ ]:
mlp = MLPClassifier()
scores = cross_validate(mlp, X_train, y_train, cv=5, 
                        scoring="roc_auc",
                        return_train_score=True)
print(f"交差検証の5回のスコア：{scores['test_score']}")
print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}")

print(f"交差検証の5回の訓練スコア：{scores['train_score']}")
print(f"交差検証の平均訓練スコア：{np.mean(scores['train_score'])}")

……ということで[ドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)を読みに行きましょう．
ハイパーパラメータの数に圧倒されるかもしれませんが，どうか諦めないでください．
いくつかのパラメータに関して簡単に見ていきます．
- `hidden_layer_sizes`：intのタプル（リストのようなもの），隠れ層（中間層）のユニット数．例えば，`(100, 50, 20)`を指定すると，中間層の数が3つで，入力に近い方から順に中間層のユニット数が100, 50, 20となる．中間層の数は「行列を掛ける」「活性化関数を噛ませる」回数に対応し，ユニット数は行列を掛けた後のベクトルの要素数に対応します．つまり，ネットワークの構造（深さと広さ）を定めるものです．
- `activation`：string，活性化関数（行列を掛けた後に作用させる非線形な関数）．ここでは，
  - `"identity"`：恒等関数 $g(x)=x$
  - `"logistic"`：ロジスティックシグモイド関数 $g(x)=1 / (1+e^{-x})$
  - `"tanh"`：ハイパボリックタンジェント $g(x)= (e^x-e^{-x}) / (e^x+e^{-x})$
  - `"relu"`：ランプ関数 $g(x)= \max (0, x)$
  
  の4つを選択できる．デフォルトは`"relu"`．
- `solver`：string，最適化手法．MLPは勾配法で最適化すると言いましたが，一口に勾配法と言っても実は色々あり，ここでは，
  - `"lbfgs"`：準ニュートン法（quasi-newton method）と呼ばれる方法の一種．ヘッセ行列（＝二回微分）を近似し，上手く用いる．
  - `"sgd"`：確率的勾配降下法（stochastic gradient descent, SGD），一回の反復に全てのデータではなく一部のデータのみを用いる．
  - `"adam"`：SGDの一種で，勾配の使い方・ステップサイズに色々と工夫が施されている．
  
  の3つを指定できる．デフォルトは`"adam"`．
- `alpha`：float，正則化項の強さ．
- `batch_size`：int， SGD・Adamにおいて，一回の反復で用いるデータの数．
- `learning_rate_init`：float，初期学習率．デフォルトは`0.001`．
- `max_iter`：int，最大反復回数．デフォルトは`200`.
- `beta_1`, `beta_2`：float，`"Adam"`のハイパーパラメータ．

ちょっと疲れたのでこの辺にしておきます．
まず，最初の`hidden_layer_sizes`と`activation`は，ネットワークの構造を決めるものなので性能に直結しそうですね．
また，最適化アルゴリズムの選択および最適化アルゴリズムのハイパーパラメータの選択も非常に重要です．
ここではとりあえず，他のハイパーパラメータはそのままで，先程試した`"relu"`以外の活性化関数を用いて交差検証で評価してみましょう．

In [ ]:
for activation in ["identity", "logistic", "tanh"]:
    print(f"活性化関数：{activation}")
    mlp = MLPClassifier(activation=activation)
    scores = cross_validate(mlp, X_train, y_train, cv=5, 
                            scoring="roc_auc")
    print(f"交差検証の5回のスコア：{scores['test_score']}")
    print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}\n")

結果が出てきました！
活性化関数の違いによって予測結果に少し影響が出ていることがわかりました．特に，活性化関数が`"logistic"`，`"tanh"`の時の性能は，活性化関数が`"identity"`，`"relu"`の時の性能と比較すると少し高く見えます．

ここで，この4種類の活性化関数のグラフを作ってみます（横軸が入力で，縦軸が関数の値）．
様々な可視化は次回行いますが，とりあえず以下のセルを動かすとグラフが出てきます．
グラフの作り方についてもう少しきちんと知りたい方は，day6_matplotlib.ipynbを見る・matplotlibで調べる等してください．

In [ ]:
# jupyter lab/notebookの時のコマンド
%matplotlib inline
import matplotlib.pyplot as plt
x = 4*np.arange(100) / 100 - 2.0
plt.plot(x, x, label="identity") # identity
plt.plot(x, 1.0 / (1.0 + np.exp(-x)), label="logistic") # logistic
plt.plot(x, np.tanh(x), label="tanh") # tanh
plt.plot(x, np.maximum(x, 0), label="relu") # relu
plt.legend(fontsize=12, loc='lower right')
plt.show()

比較的結果の悪かった`"identity"`と`"relu"`は非有界（=関数の値がどこまでも大きくなる or 小さくなる）ですが，（お世辞にも良い結果ではないですが）`"logistic"`と`"tanh"`は有界（前者は$(0, 1)$，後者は$(-1, 1)$）です．
そのため，以下のようなことが起こっているのではないかと考えられます：**活性化関数に通す前の中間層の値が非常に大きくなってしまっているのではないか？**

活性化関数に通す前の値は，「入力ベクトルと重み行列の積」によって計算されます．
したがって，特徴ベクトルの値が大きい場合，中間層の値が非常に大きくなってしまう可能性があります．
特徴ベクトルの値をちょっと思い出してみます．
one-hotエンコーディングによって作られた部分は0か1でした．
各サイズで売れた数を表す部分は，かなり大きな数字が入っていたような記憶があると思います．
ちょっと見てみましょう．

In [ ]:
print(d_train)
print(np.max(X_train))

最大で5ケタの大きな値が入っており，先程述べていたことが起こっている可能性が高そうに思えてきます．
また，そもそも，特徴の値の最大値と最小値の範囲が，各特徴毎に大きく異なるというのは問題となることがあります．
線形モデルの例を考えてみます．
one-hotエンコーディングされたある特徴（たとえば，`"ever_married"`）の重みを$w_j$，数値として与えられたある特徴（たとえば，`id`）に関する重みを$w_i$とし，$w_j=w_i=0.01$であるとします．
このとき，one-hotエンコーディングされたある特徴の重み$w_j=0.01$であることに大きな問題はなさそうです：$x_j$は最大でも$1$なので，$x_j=0$の場合と比較しても予測の値が$0.1$程度増えるだけです．
一方で，売れた数に関する重み **$w_i=0.01$は問題が起こります**：$x_i$の値が，例えば上で出ている最大値`72940.0`のとき，$x_i=0$の場合と比較して，**予測の値がおよそ729も増えてしまいます**．
つまり，各特徴に対応するパラメータ（上の例では，$w_j$や$w_i$）のスケールは，その特徴のスケールに応じて適切なものになっていなければなりません．

[sklearnのニューラルネットワークに関する解説](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use)を見てみましょう．
次のように書いてあります：

> "Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. For example, scale each attribute on the input vector X to [0, 1] or [-1, +1], or standardize it to have mean 0 and variance 1. Note that you must apply the same scaling to the test set for meaningful results. You can use StandardScaler for standardization."

ようするに， **MLPは特徴のスケールに影響を受けやすいので，そこらへんはどうにかしなさい（例えば，各特徴の値が$[0, 1]$や$[-1, 1]$の範囲に入るようにであったり，平均が0で分散が1になるようにしなさい）** と言っています．
このように，学習アルゴリズムに入れる前に，特徴量に対して何らかの変換を施すことを **前処理（preprocessing）** と言います（one-hotエンコーディングも前処理の一部であると言えるでしょう）．

## `sklearn.preprocessing`：sklearnの機能を使って特徴量のスケールを揃える

さて，それでは上で言われているように，特徴ベクトルのスケールを揃えてみましょう．
今回は各特徴量を`[0, 1]`の範囲に（=最小値が0で最大値が1になるように）スケーリングします．
numpyの演算を使うことでも比較的簡単にできますが，ここではsklearnの機能を使うことにします．

sklearnには`sklearn.preprocessing`というモジュールがあり，このモジュールの中に様々な前処理のための関数・クラスが用意されています．
今回は，**最大値と最小値を揃えるスケーリング**を行いたいわけですが，これは`sklearn.preprocessing`の中の`MinMaxScaler`によって行うことができます．

そこで，今回のQuizです．

### Quiz 1
[MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)を使って，各特徴の値を[0,1]の範囲にスケーリングした訓練データ`X_train_scaled`を作成し，`X_train_scaled`を用いて`MLPClassifier()`の交差検証を行してください．

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 以下を埋める


X_train_scaled =  # ここを埋める

# モデルを作成し，交差検証で評価
mlp = MLPClassifier()
scores = cross_validate(mlp, X_train_scaled, y_train, cv=5, 
                        scoring="roc_auc",
                        return_train_score=True)
print(f"交差検証の5回のスコア：{scores['test_score']}")
print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}")

print(f"交差検証の5回の訓練スコア：{scores['train_score']}")
print(f"交差検証の平均訓練スコア：{np.mean(scores['train_score'])}")

スコアが改善しました！
しかもまだ細かい交差検証はしていませんから，更に性能の向上を見込めます．

交差検証をして良いパラメータを定めてみましょう．
前回の復習も兼ねて，続けてクイズです．

### Quiz 2
`X_train_scaled`を訓練データの入力として，`MLPClassifier`のハイパーパラメータを`GridSearchCV`によってチューニングしてください．
ここで，以下の2つのパラメータを次で示す範囲でチューニングすること：
- `hidden_layer_sizes`：(1)中間層が一つでユニット数が100 (2)中間層が2つでユニット数がそれぞれ80
- `activation`：(1) "logistic" (2) "tanh" (3) "relu"

また，計算時間の削減のため反復数`max_iter`は100とし，乱数の初期SEEDを定める`random_state`は765としてください．交差検証の分割数は5，スコア関数は`"roc_auc"`とすること．
そして最後に，最も良かったハイパーパラメータの組と，その時の交差検証のスコアを`print`してください．

In [ ]:
mlp = # ここを埋める
params_grid = {} # ここを埋める
# 以下を埋める

最も良かったハイパーパラメータとその時のスコアについて，私の環境では以下の結果が得られました：

```Python
{'activation': 'tanh', 'hidden_layer_sizes': (100,)}

0.8328010747254228
```

この結果で得られたベストなモデルを使って予測結果を提出すると良いかもしれません．
その際，**テストデータの変換を行うのを忘れる・変換の仕方を誤る**と，ひどいスコアとなってしまうためご注意ください．
なお，上のチューニングはかなり粗く・雑に行っているので，より丁寧に行うことでより良いスコアが出る可能性は高いです．

## まとめ
- 多層パーセプトロン（MLP）：ニューラルネットワークの一種．（非線形な活性化関数を使った場合）非線形なモデルで，特徴ベクトルのスケールに影響を強く受ける．
- 前処理のための関数やクラスは`sklearn.preprocessing`にある．最大値と最小値のスケーリングは`MinMaxScaler`で行うことができる．

今回用いた`MinMaxScaler`以外にも前処理のクラス・関数は多数あるので，それらについても調べてみる・使ってみると良いかもしれません．
また，具体的に名前を列挙することはしませんが，非線形なモデルは`MLPClassifier`以外にも多数あります．
その中には，特徴ベクトルのスケールの影響を受けにくく，使いやすいモデルもあります．
また，興味深いことに，カテゴリカル変数をそのまま扱える手法もあります（残念ながらsklearnの実装ではそうなっておらず，その手法に特化したライブラリを使う必要がありますが）．
「どのような特徴を作るか」「どのような変換を施すか」「どのようなモデルを使うか」「どのようなハイパーパラメータにするか」を考えると，できることは山ほどあるので，是非色々試してみてください．

さらに，ここ10年弱の深層学習ブームにより，ニューラルネットワークのライブラリが多数開発されています．
それらを用いることでより自由に・柔軟にニューラルネットワークを（MLPを）構築することができます．
それらについても興味があれば調べてみると良いかもしれません．

今回は，予測モデルを構築する前の処理として，スケーリングを取り上げました．
それ以外にも，例えば
- 特徴選択（feature selection）：元の特徴量からいくつかの特徴量を選ぶ（特徴選択で作られた低次元ベクトルは，元の特徴ベクトルの部分ベクトル）
- 特徴抽出（feature extraction）：元の特徴量を用いて新しく（低次元の）特徴量を作り出す（元の特徴ベクトルの部分ベクトルとは限らない）

等の処理を行うと，性能が向上することもあります．
これらはsklearnでは[sklearn.feature_selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)や[sklearn.decomposition](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition)に実装されています．
余裕がある方は試してみると良いかもしれません．

## 付録：EDA（Exploratory Data Analysis, 探索的データ分析）

ここまで，いくつかの学習モデルと様々な処理（one-hotエンコーディング，欠損値処理，交差検証，ハイパーパラメータ探索など）について見てきました．紹介した以外にも数多くのモデルや処理が存在し，その中で「うまくいく」方法を見つけることがコンペでよいスコアを出すためには重要です．しかし，片っ端からありとあらゆる手法を試すのは時間的な制約（コンペの開催期間）から考えても難しいでしょう．したがって，コンペにおいては「うまくいきそうな」手法を優先的に試すことになると思います．このときの手法選択の手がかりをつかむための手法として **EDA（Exploratory Data Analysis, 探索的データ分析）** が挙げられます．

EDAでは，与えられたデータの特徴をつかみます．具体的には，

- どのようなデータ（種類，数）が与えられているか
- 各データの分布，例えば最大値や最小値，平均値や分散はどうなっているか
- 欠損値はどれほど存在するか
- 説明変数どうし，あるいは説明変数と目的変数の間に相関はあるか

といった情報を

- 棒グラフ
- 箱ひげ図
- 散布図
- 折れ線グラフ
- ヒートマップ
- ヒストグラム

などの可視化手法を用いて可視化し，データに関する情報を得ることを狙いとします．これらの情報は，どのようなモデルや手法なら「うまくいきそうか」，またどのような処理を行うべきかの参考にすることができます．

EDAは，適宜必要なコードを自分で書き行うこともできますが，自分で0からコードを書くのは大変かもしれません．
最も楽な方法の一つとして，`pandas`の`pandas_profiling`（[ドキュメント](https://pandas-profiling.ydata.ai/docs/master/)）のような既存の可視化ツールを使用することが挙げられます．
以下で、`pandas_profiling`の使用方法について説明します．

例えば訓練データの情報を見たいとき，以下のように実行できます．

（セルの実行中に`Proceed (y/n)?`のように聞かれた場合，`y`と回答してください．）

In [ ]:
# Google Colabを使っている場合はこのセルを実行する（pandas_profiling, markupsafeのバージョンを更新）
# 自身のPython環境を用いている場合、適宜インストールを行う
!pip uninstall pandas_profiling
!pip install -U pandas_profiling
!pip install markupsafe==2.0.1

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(pd.read_csv("data/train.csv"))

上のセルを実行すると，様々な情報が表示されると思います．こうした情報をもとにモデルや手法を探すことで，より良いスコアを狙うことができます．一般にコンペにおいては正解というものはありません．データと自分の直感とをすり合わせ，思い思いのモデルを作成してみてください．

参考文献：「Kaggleで勝つデータ分析の技術」（2019，門脇大輔ほか，技術評論社）

## Answer

### Quiz 1

`from sklearn.preprocessing import MinMaxScaler`で使えるようになります．
`MinMaxScaler`は，標準では[0,1]にスケーリングするため，特に引数の設定をすることなくインスタンスを作成します．
ドキュメントを読んでみると分かりますが，`fit`メソッドで各特徴量の最大値と最小値を計算し，`transform`メソッドで変換を施した行列を返します．
そのため，以下のようになります．

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 以下を埋める
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train) # ここを埋める

# モデルを作成し，交差検証で評価
mlp = MLPClassifier()
scores = cross_validate(mlp, X_train_scaled, y_train, cv=5, 
                        scoring="roc_auc",
                        return_train_score=True)
print(f"交差検証の5回のスコア：{scores['test_score']}")
print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}")

print(f"交差検証の5回の訓練スコア：{scores['train_score']}")
print(f"交差検証の平均訓練スコア：{np.mean(scores['train_score'])}")

### Quiz 2

In [ ]:
mlp = MLPClassifier(max_iter=100, random_state=765) # ここを埋める
params_grid = {
    "hidden_layer_sizes": [(100, ), (80, 80)],
    "activation": ["logistic", "tanh", "relu"],} # ここを埋める
# 以下を埋める
cv = GridSearchCV(mlp, param_grid=params_grid, cv=5, scoring="roc_auc")
cv.fit(X_train_scaled, y_train)
print(cv.best_params_) # 最も良かったハイパーパラメータを見てみる
print(cv.best_score_) # 最も良かったハイパーパラメータの時のスコアを見てみる．